In [69]:
import pandas as pd
import joblib   
import numpy as np
from SimulateDay import scale_data, predict_action, get_stock_data
import xgboost as xgb


def stock_market_simulation(model, initial_cash, days, stock, stock_name, monthly_injection=0):
    # Add Taxes and Fees
    cash = initial_cash
    invested = cash
    shares_held = 0
    portfolio_value = []
    scaled = scale_data(stock)
    modelDecisionDf = pd.DataFrame(
        columns=['Stock Name', 'Day', 'Action', 'Cash', 'Shares Held', 'Portfolio Value'])

    days = min(days, len(stock))
    for i in range(days):
        stock_price = stock['Close'].iloc[i]
        strategy = predict_action(scaled.iloc[i].to_dict(), model)
        if i // 30 == 0:
            cash += monthly_injection
            invested += monthly_injection
        if strategy == 'Buy' and cash >= stock_price:
            # Buy one share if cash is sufficient
            cash -= stock_price
            shares_held += 1
            print(f"Day {i}: Bought 1 share at {stock_price}, Cash left: {cash}")

        elif strategy == 'Buy' and cash < stock_price:
            shares_held += cash / stock_price
            cash = 0
            print(
                f"Day {i}: Bought {cash / stock_price} shares at {stock_price}, Cash left: {cash}")

        elif strategy == 'Sell' and shares_held > 0:
            # Sell one share if we hold any
            cash += stock_price
            shares_held -= 1
            print(f"Day {i}: Sold 1 share at {stock_price}, Cash: {cash}")

        elif cash == 0:
            injection = input(
                f'Your current portfolio value is {portfolio_value[i-1]}. The stock price is {stock_price}. Enter the amount of cash you want to inject: ')
            if injection:
                cash += float(injection)
                invested += float(injection)
                print(f"Day {i}: Cash injected: {injection}, Cash: {cash}")
                if cash < stock_price:
                    break

        # Calculate the total portfolio value (cash + stock holdings)
        portfolio_value_at_time = cash + (shares_held * stock_price)
        portfolio_value.append(portfolio_value_at_time)

        new_row = pd.DataFrame({
            'Stock Name': [stock_name],
            'Day': [i],
            'Date': [stock['Date'].iloc[i]],
            'Action': [strategy],
            'Stock Price': [stock_price],
            'Cash': [cash],
            'Shares Held': [shares_held],
            'Portfolio Value': [portfolio_value_at_time]
        })
        modelDecisionDf = pd.concat(
            [modelDecisionDf, new_row], ignore_index=True)

    # Final results
    final_portfolio_value = cash + (shares_held * stock['Close'].iloc[-1])
    print(f'Total cash invested: {invested}')
    print(f'Stock {stock["Symbol"].iloc[0]}')
    print(f"Final Portfolio Value: {final_portfolio_value}")
    print(f"Cash: {cash}, Shares held: {shares_held}")
    return modelDecisionDf, cash
    # return portfolio_value, final_portfolio_value


In [70]:

specific_model_decisions = pd.read_csv('data/specific_model_decisions.csv')
# general_model_decisions = pd.read_csv('data/general_model_decisions.csv')

general_model = xgb.Booster()
general_model.load_model('models/all_stocks_incremental_model.pkl') 

specific_model_decisions['Action'].value_counts()
# general_model_decisions['Action'].value_counts()

Action
Sell    256
Buy     144
Hold    103
Name: count, dtype: int64

In [104]:
stock_data = get_stock_data('MSFT')
stock_data = stock_data[stock_data['Date'] >= '2021-01-22']
specific_model = joblib.load('OptimalActionModel.pkl')
specific_model0 = joblib.load('models/MSFT_model.pkl')
specific_decisions,_ = stock_market_simulation(specific_model0,10000,1000,stock_data,'MSFT',1000)
# decision,_ = stock_market_simulation(general_model,10000,1000,stock_data,'AAPL',1000)

/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:202: FutureWarning:

The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError



Day 0: Bought 1 share at 225.9499969482422, Cash left: 10774.050003051758
Day 1: Bought 1 share at 229.52999877929688, Cash left: 11544.520004272461
Day 2: Bought 1 share at 232.3300018310547, Cash left: 12312.190002441406
Day 3: Bought 1 share at 232.8999938964844, Cash left: 13079.290008544922
Day 4: Bought 1 share at 238.92999267578125, Cash left: 13840.36001586914
Day 5: Bought 1 share at 231.9600067138672, Cash left: 14608.400009155273
Day 6: Bought 1 share at 239.6499938964844, Cash left: 15368.750015258789
Day 7: Bought 1 share at 239.5099945068359, Cash left: 16129.240020751953


/var/folders/75/l95mxw1j5w9ddljb82_6_m_40000gn/T/ipykernel_90530/3205775490.py:67: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Day 8: Bought 1 share at 243.0, Cash left: 16886.240020751953
Day 9: Bought 1 share at 242.0099945068359, Cash left: 17644.230026245117
Day 10: Bought 1 share at 242.1999969482422, Cash left: 18402.030029296875
Day 11: Bought 1 share at 242.47000122070312, Cash left: 19159.560028076172
Day 12: Bought 1 share at 243.77000427246097, Cash left: 19915.79002380371
Day 13: Bought 1 share at 242.82000732421875, Cash left: 20672.970016479492
Day 14: Bought 1 share at 244.4900054931641, Cash left: 21428.480010986328
Day 15: Bought 1 share at 244.9900054931641, Cash left: 22183.490005493164
Day 16: Bought 1 share at 243.6999969482422, Cash left: 22939.790008544922
Day 17: Bought 1 share at 244.1999969482422, Cash left: 23695.59001159668
Day 18: Sold 1 share at 243.7899932861328, Cash: 24939.380004882812
Day 19: Sold 1 share at 240.97000122070312, Cash: 26180.350006103516
Day 20: Sold 1 share at 234.5099945068359, Cash: 27414.86000061035
Day 21: Sold 1 share at 233.27000427246097, Cash: 28648.130

In [79]:
specific_decisions.tail()

,Stock Name,Day,Action,Cash,Shares Held,Portfolio Value,Date,Stock Price
929,AMZN,929,Buy,15609.127502,146,42584.086700,2024-10-02,184.759995
930,AMZN,930,Buy,15427.167496,147,42175.288483,2024-10-03,181.960007
931,AMZN,931,Hold,15427.167496,147,42844.136688,2024-10-04,186.509995
932,AMZN,932,Sell,15607.967499,146,42004.767944,2024-10-07,180.800003
933,AMZN,933,Hold,15607.967499,146,42642.787231,2024-10-09,185.169998


In [80]:
general_decisions,_ = stock_market_simulation(general_model,10000,1000,stock_data,'AMZN',1000)


/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:202: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  stock_df['Best_Return_Window'] = stock_df[['1_Day_Return', '5_Day_Return',
/var/folders/75/l95mxw1j5w9ddljb82_6_m_40000gn/T/ipykernel_90530/3205775490.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modelDecisionDf = pd.concat(


Day 1: Bought 1 share at 164.6999969482422, Cash left: 11835.300003051758
Day 2: Bought 1 share at 166.30650329589844, Cash left: 12668.99349975586
Day 3: Sold 1 share at 161.62899780273438, Cash: 13830.622497558594
Day 8: Bought 1 share at 165.62649536132812, Cash left: 18664.996002197266
Day 11: Bought 1 share at 166.14700317382812, Cash left: 21498.848999023438
Day 12: Sold 1 share at 165.25, Cash: 22664.098999023438
Day 13: Sold 1 share at 164.32899475097656, Cash: 23828.427993774414
Day 14: Sold 1 share at 163.10650634765625, Cash: 24991.53450012207
Day 50: Bought 1 share at 161.1909942626953, Cash left: 39830.343505859375
Day 54: Bought 1 share at 168.96949768066406, Cash left: 39661.37400817871
Day 55: Sold 1 share at 170.0, Cash: 39831.37400817871
Day 56: Bought 1 share at 166.64999389648438, Cash left: 39664.72401428223
Day 57: Sold 1 share at 168.95449829101562, Cash: 39833.67851257324
Day 58: Sold 1 share at 169.9720001220703, Cash: 40003.65051269531
Day 59: Bought 1 share a

In [81]:
general_decisions.tail()

,Stock Name,Day,Action,Cash,Shares Held,Portfolio Value,Date,Stock Price
929,AMZN,929,Sell,39903.547997,0,39903.547997,2024-10-02,184.759995
930,AMZN,930,Sell,39903.547997,0,39903.547997,2024-10-03,181.960007
931,AMZN,931,Sell,39903.547997,0,39903.547997,2024-10-04,186.509995
932,AMZN,932,Sell,39903.547997,0,39903.547997,2024-10-07,180.800003
933,AMZN,933,Hold,39903.547997,0,39903.547997,2024-10-09,185.169998


In [83]:
stock_df = pd.read_csv('data/sp500_stocks.csv')
stock_df['Smoothed_Close'] = stock_df['Close'].rolling(window=5).mean()


In [92]:
import pandas as pd
import numpy as np

# Use a smoothed version of 'Close' to detect peaks and troughs
stock_df['Smoothed_Close'] = stock_df['Close'].rolling(window=5).mean()

# Find local minima (buy points) and local maxima (sell points)
# Local minima (buy points)
stock_df['Buy_Signal'] = (stock_df['Smoothed_Close'].shift(1) > stock_df['Smoothed_Close']) & (stock_df['Smoothed_Close'].shift(-1) > stock_df['Smoothed_Close'])

# Local maxima (sell points)
stock_df['Sell_Signal'] = (stock_df['Smoothed_Close'].shift(1) < stock_df['Smoothed_Close']) & (stock_df['Smoothed_Close'].shift(-1) < stock_df['Smoothed_Close'])

# Initialize 'Optimal_Action' column with 'Hold'
stock_df['Optimal_Action'] = 'Hold'

# Assign 'Buy' where Buy_Signal is True
stock_df.loc[stock_df['Buy_Signal'], 'Optimal_Action'] = 'Buy'

# Assign 'Sell' where Sell_Signal is True
stock_df.loc[stock_df['Sell_Signal'], 'Optimal_Action'] = 'Sell'

# Clean up: drop the temporary signals if needed
stock_df.drop(['Buy_Signal', 'Sell_Signal', 'Smoothed_Close'], axis=1, inplace=True)

# Check the resulting DataFrame
stock_df[['Date', 'Close', 'Optimal_Action']].tail(50)




,Date,Close,Optimal_Action
1868595,2024-07-29,181.830002,Hold
1868596,2024-07-30,181.830002,Hold
1868597,2024-07-31,180.039993,Hold
1868598,2024-08-01,182.419998,Hold
1868599,2024-08-02,180.589996,Sell
1868600,2024-08-05,174.820007,Buy
1868601,2024-08-06,185.289993,Hold
1868602,2024-08-07,184.770004,Hold
1868603,2024-08-08,188.300003,Hold
1868604,2024-08-09,185.250000,Hold
